In [3]:
from modelzipper.tutils import *
from inference.utils.babilong.prompts import DEFAULT_PROMPTS, DEFAULT_TEMPLATE, get_formatted_input, LongMIT_DEFAULT_TEMPLATE
from datasets import Dataset, DatasetDict

data_dir = "/mnt/petrelfs/tangzecheng/remote_bucket/zecheng/data/babilong/generated_tasks"

tasks = ["qa2", "qa3"]



begin to read data from /mnt/petrelfs/tangzecheng/remote_bucket/zecheng/data/babilong/generated_tasks/qa2/16k.pkl | file size: 61.86 MB | file type: pkl
begin to read data from /mnt/petrelfs/tangzecheng/remote_bucket/zecheng/data/babilong/generated_tasks/qa2/32k.pkl | file size: 124.96 MB | file type: pkl
begin to read data from /mnt/petrelfs/tangzecheng/remote_bucket/zecheng/data/babilong/generated_tasks/qa2/4k.pkl | file size: 15.07 MB | file type: pkl
begin to read data from /mnt/petrelfs/tangzecheng/remote_bucket/zecheng/data/babilong/generated_tasks/qa2/64k.pkl | file size: 250.01 MB | file type: pkl
begin to read data from /mnt/petrelfs/tangzecheng/remote_bucket/zecheng/data/babilong/generated_tasks/qa2/8k.pkl | file size: 30.67 MB | file type: pkl
begin to read data from /mnt/petrelfs/tangzecheng/remote_bucket/zecheng/data/babilong/generated_tasks/qa3/16k.pkl | file size: 63.08 MB | file type: pkl
begin to read data from /mnt/petrelfs/tangzecheng/remote_bucket/zecheng/data/babil

In [24]:

for task in tasks:
    all_content = []
    task_dir = os.path.join(data_dir, task)
    all_files = os.listdir(task_dir)
    for f in all_files:
        content = auto_read_data(os.path.join(task_dir, f))
        for item in content:
            item['task'] = task
            item['f'] = f.split('.')[0]
        all_content.extend(content)

    anlysis_data = []

    for sample in all_content:
        prompt_cfg = {
            'instruction': DEFAULT_PROMPTS[task]['instruction'],
            'examples': DEFAULT_PROMPTS[task]['examples'],
            'post_prompt': DEFAULT_PROMPTS[task]['post_prompt'],
            'template': LongMIT_DEFAULT_TEMPLATE,
            'chat_template': True,
        }
        prompt_name = [f'{k}_yes' if prompt_cfg[k] else f'{k}_no' for k in prompt_cfg if k != 'template']
        prompt_name = '_'.join(prompt_name)

        target, context, question = sample['target'], sample['input'], sample['question']
        input_text = get_formatted_input(
            context, question, prompt_cfg['examples'],
            prompt_cfg['instruction'], prompt_cfg['post_prompt'],
            template=prompt_cfg['template']
        )

        all_clues = sample['reference'].tolist()
        attack_clues = list(set(sample['facts'].tolist()) - set(all_clues))

        anlysis_data.append({
            "prompt": input_text,
            "message": [
                {"role": "user", "content": input_text}, 
                {"role": "assistant", "content": target}
            ],
            "all_clues": all_clues,
            "attack_clues": attack_clues,
            "length": sample['f']
        })

    anlysis_data = Dataset.from_list(anlysis_data)
    dataset_dict = DatasetDict({
        "train": anlysis_data
    })

    dataset_dict.save_to_disk(f"/mnt/petrelfs/tangzecheng/local_data/analysis_multihop_data/babilong_{task}")

begin to read data from /mnt/petrelfs/tangzecheng/remote_bucket/zecheng/data/babilong/generated_tasks/qa2/16k.pkl | file size: 61.86 MB | file type: pkl
begin to read data from /mnt/petrelfs/tangzecheng/remote_bucket/zecheng/data/babilong/generated_tasks/qa2/32k.pkl | file size: 124.96 MB | file type: pkl
begin to read data from /mnt/petrelfs/tangzecheng/remote_bucket/zecheng/data/babilong/generated_tasks/qa2/4k.pkl | file size: 15.07 MB | file type: pkl
begin to read data from /mnt/petrelfs/tangzecheng/remote_bucket/zecheng/data/babilong/generated_tasks/qa2/64k.pkl | file size: 250.01 MB | file type: pkl
begin to read data from /mnt/petrelfs/tangzecheng/remote_bucket/zecheng/data/babilong/generated_tasks/qa2/8k.pkl | file size: 30.67 MB | file type: pkl


Saving the dataset (3/3 shards): 100%|██████████| 5000/5000 [00:02<00:00, 1913.51 examples/s]


begin to read data from /mnt/petrelfs/tangzecheng/remote_bucket/zecheng/data/babilong/generated_tasks/qa3/16k.pkl | file size: 63.08 MB | file type: pkl
begin to read data from /mnt/petrelfs/tangzecheng/remote_bucket/zecheng/data/babilong/generated_tasks/qa3/32k.pkl | file size: 125.73 MB | file type: pkl
begin to read data from /mnt/petrelfs/tangzecheng/remote_bucket/zecheng/data/babilong/generated_tasks/qa3/4k.pkl | file size: 16.1 MB | file type: pkl
begin to read data from /mnt/petrelfs/tangzecheng/remote_bucket/zecheng/data/babilong/generated_tasks/qa3/64k.pkl | file size: 251.4 MB | file type: pkl
begin to read data from /mnt/petrelfs/tangzecheng/remote_bucket/zecheng/data/babilong/generated_tasks/qa3/8k.pkl | file size: 31.66 MB | file type: pkl


Saving the dataset (3/3 shards): 100%|██████████| 5000/5000 [00:02<00:00, 1870.73 examples/s]
